In [2]:
import tkinter as tk
from tkinter import filedialog, messagebox, scrolledtext

class Linecounter:
    def __init__(self, file_name):
        self.file_name = file_name
        self.line = []
        self.ip_address = []

    def read(self):
        with open(self.file_name, "r") as f:
            self.line = f.readlines()

    def fetch_ip_add(self):
        self.ip_address = [x.split(" ")[0] for x in self.line]
        return self.ip_address

    def ip_add_filter(self, filter_value):
        try:
            return [x for x in self.ip_address if int(x.split(".")[0]) < filter_value]
        except ValueError:
            return []

    def ratio(self, filter_value):
        if len(self.ip_address) == 0:
            return 0
        return len(self.ip_add_filter(filter_value)) / len(self.ip_address)

# GUI Part
class IPAnalyzerApp:
    def __init__(self, root):
        self.root = root
        self.root.title("IP Address Analyzer")
        self.root.geometry("600x550")

        self.filename = None
        self.lc = None

        self.load_btn = tk.Button(root, text="Load Log File", command=self.load_file)
        self.load_btn.pack(pady=10)

        self.ip_display = scrolledtext.ScrolledText(root, height=12)
        self.ip_display.pack(padx=10, pady=10, fill="both", expand=True)

        self.filter_frame = tk.Frame(root)
        self.filter_frame.pack(pady=5)

        self.filter_label = tk.Label(self.filter_frame, text="Filter (first octet < ):")
        self.filter_label.pack(side=tk.LEFT, padx=5)

        self.filter_entry = tk.Entry(self.filter_frame, width=10)
        self.filter_entry.insert(0, "20")
        self.filter_entry.pack(side=tk.LEFT, padx=5)

        self.show_ip_btn = tk.Button(root, text="Show All IPs", command=self.show_ips)
        self.show_ip_btn.pack(pady=5)

        self.show_filtered_btn = tk.Button(root, text="Show Filtered IPs", command=self.show_filtered_ips)
        self.show_filtered_btn.pack(pady=5)

        self.ratio_label = tk.Label(root, text="Ratio: -")
        self.ratio_label.pack(pady=10)

    def load_file(self):
        self.filename = filedialog.askopenfilename(title="Select log file", filetypes=(("Text files", "*.txt"), ("All files", "*.*")))
        if not self.filename:
            return
        try:
            self.lc = Linecounter(self.filename)
            self.lc.read()
            self.lc.fetch_ip_add()
            messagebox.showinfo("Success", "File loaded and IPs fetched!")
        except Exception as e:
            messagebox.showerror("Error", f"Failed to load file: {str(e)}")

    def show_ips(self):
        if not self.lc:
            messagebox.showwarning("Warning", "Please load a file first.")
            return
        self.ip_display.delete("1.0", tk.END)
        for ip in self.lc.ip_address:
            self.ip_display.insert(tk.END, ip + "\n")
        self.ratio_label.config(text="Ratio: -")

    def show_filtered_ips(self):
        if not self.lc:
            messagebox.showwarning("Warning", "Please load a file first.")
            return
        try:
            filter_value = int(self.filter_entry.get())
        except ValueError:
            messagebox.showerror("Error", "Please enter a valid integer for filter.")
            return
        filtered_ips = self.lc.ip_add_filter(filter_value)
        self.ip_display.delete("1.0", tk.END)
        for ip in filtered_ips:
            self.ip_display.insert(tk.END, ip + "\n")
        ratio_val = self.lc.ratio(filter_value)
        self.ratio_label.config(text=f"Ratio: {ratio_val:.2f}")

# Run the app
if __name__ == "__main__":
    root = tk.Tk()
    app = IPAnalyzerApp(root)
    root.mainloop()
